# Genera un archivo csv nuevo con solo las filas que son en ingles

In [1]:
import pandas as pd
from time import time
import gc

#Comienza a leer el tiempo
cabecera = pd.read_csv('steam_reviews.csv',',',nrows=0)#Lee 1 vez el archivo obtener la cabecera
only_english = pd.DataFrame(columns=cabecera.columns.values)#Crea un nuevo DF
rows = 0#Cantidad de filas leidas.
chunk_size = 1000000#Tamaño de filas a leer 
fileN = 1#Veces dividido
for chunk in pd.read_csv('steam_reviews.csv',',',chunksize=chunk_size):
    #chunk.to_csv('files/steam_reviews-' + str(fileN)+'.csv', index=False)#Crea un nuevo archivo con los datos leidos hasta ahora.
    fileN +=1#Suma 1 a la cantidad de archivos creados
    rows+=chunk_size#Suma la cantidad de filas leidas
    only_english = only_english.append(chunk[chunk['language'] == 'english'],ignore_index = True)#Agrega al nuevo csv solo las filas en inglesh

print("Cantidad de diviciones: ",fileN)
print("Total de columnas iteradas: ",rows)
tiempoFinal = time()
tiempoTotal = tiempoFinal - tiempoInicial
print("Tiempo total en leer los datos: ", tiempoTotal)
only_english.to_csv('english-steam-reviews.csv',index=False)#Genera el nuevo archivo
del [[only_english]]
gc.collect()
tiempoFinal = time()
tiempoTotal = tiempoFinal - tiempoInicial
print("Tiempo total de ejecucion: ", tiempoTotal)


Cantidad de diviciones:  23
Total de columnas iteradas:  22000000
Tiempo total en leer los datos:  193.2555136680603
Tiempo total de ejecucion:  304.86002492904663


## Comienza analisis y tratamiento de los 2 dataset

In [1]:
import pandas as pd
from time import time
import gc


tiempoInicialCruzamiento = time()

tiempoInicialLectura = time()

#carga los archivos
gfdf = pd.read_csv('games-features.csv', delimiter=",")
esrdf = pd.read_csv('english-steam-reviews.csv', delimiter=",")

#Printea cuanto tiempo tomo esa accion.
tiempoFinalLectura = time() - tiempoInicialLectura
print("tiempo de demora en leer los archivos: ", tiempoFinalLectura)

#Clasification Review 
tiempoInicialClasificacion = time()

from numpy import string_

goodW = ['Great','Good','Favorite','Exciting','Fine','Best','Favourite','Excellent','Nice','Popular','Splendid'
,'Fascinating','Wonderful','Beautiful','Sweet','Magnificent','Fantastic','Complete','Balanced','Worthy','Fun']

badW = ['Terrible','Terribly','Suck','Sucks','Poor','Poorest','Stupid','Shit','Bad','Worst','Worse','Disgusting','Nasty','Appalling','Hopeless'
,'Horrible','Fool','Foolish','Idiot','Fatal','Expensive','Vulgar','Useless','Unconventional','Ugly','Troublesome','Annoying','Bothering']

def contarBuenas(oracion):
    cont = 0
    for i in range(len(goodW)):
        cont += oracion.count(goodW[i].lower())
    return cont

def contarMalas(oracion):
    cont = 0
    for i in range(len(badW)):
        cont += oracion.count(badW[i].lower())
    return cont   

def clasificarRev(contBuenas, contMalas):
    if contBuenas > contMalas:
        return 'Positive'
    elif contBuenas < contMalas:
        return 'Negative'
    else:
        return 'Neutral'

def agregarDato(rev):
    rev = str(rev)
    contGood = contarBuenas(rev.lower())
    contBad = contarMalas(rev.lower())  
    result = clasificarRev(contGood, contBad)
    return result

esrdf['clasification_reviews'] = esrdf['review'].transform(agregarDato)
esrdf

tiempoTotalClasificacion = time - tiempoInicialClasificacion
print("Tiempo total de clasificacion = ", tiempoTotalClasificacion)

tiempoCambioFechas = time()

from datetime import datetime

def changeDate(timestap):
    return datetime.fromtimestamp(timestamp)

esrdf['timestamp_created'] = esrdf['timestamp_created'].transform(changeDate)
esrdf['timestamp_updated'] = esrdf['timestamp_updated'].transform(changeDate)
tiempoFinalCambioFechas = time - tiempoCambioFechas
print("Tiempo total de cambio de fechas = ", tiempoFinalCambioFechas)

tiempo de demora en leer los archivos:  61.190919399261475
Cantidad de juegos que tienen Reviews =  315
Tiempo que tomo reducir el archivo GameFeatures:  11.98086929321289
Tiempo de escritura de nuevo GameFeaturesCSV =  0.05996847152709961
Tiempo total de ejecucion para tratar y guardar GameFeatures =  12.04083776473999
Cantidad de juegos disponibles en GameFeaturesFinal:  142
Tiempo que tomo generar nuevo data frame con las reviews de los juegos que existen en game features =  372.47310996055603
Tiempo que tomo escribir el nuevo steam reviews =  67.56201195716858
Tiempo total de tratamiento a steam reviews =  440.0351219177246
Tiempo total reduccion ambos archivos =  452.0769603252411
Tiempo total de ejecucion:  513.2678797245026


# Reduccion de Archivos

In [ ]:
esrdf_columns = esrdf.columns#Obten las columnas del steamReviews

#Empieza a contar un nuevo tiempo, de cuanto tomo reducir el gameFeatures.
tiempoInicialReduccion = time()

#Arreglo auxiliar que obtendra las id unicas de los juegos que hay en steam reviews
idarray = list()
#Agrega todas las id unicas de los juegos de steam reviews
for i in esrdf['app_id']:
    if( not (i in idarray)):#Si alguna id no esta en el arreglo de ids 
        idarray.append(i)#Agregala

print("Cantidad de juegos que tienen Reviews = ", len(idarray))

#Crea un nuevo dataframe final para el GameFeatures
newGameFeatures = pd.DataFrame(columns=gfdf.columns)
insertedID = []
for i in idarray:
    row = gfdf.query("ResponseID == "+str(i))
    newGameFeatures = newGameFeatures.append(row)
    
newGameFeatures = newGameFeatures.drop_duplicates(subset=['ResponseID'])

#Columnas que se eliminaran
toDelete = ['DemoCount','DeveloperCount', 'MovieCount',
                    'PackageCount', 'PublisherCount', 'Background',
                    'DRMNotice', 'HeaderImage', 'LegalNotice',
                    'SupportedLanguages', 'Website', 'ExtUserAcctNotice']

for to in toDelete:
    del newGameFeatures[to]
    
tiempoFinalReduccion1 = time() - tiempoInicialReduccion
print("Tiempo que tomo reducir el archivo GameFeatures: ", tiempoFinalReduccion1)

#Crea un nuevo archivo csv y muestra cuanto tiempo tomo eso.
tiempoInicialEscribirGameFeaturesCSV = time()
newGameFeatures.to_csv('final-game-features.csv', index=False)#Escribe el archivo
tiempoFinalEscribirGameFeaturesCSV = time() - tiempoInicialEscribirGameFeaturesCSV
print("Tiempo de escritura de nuevo GameFeaturesCSV = ", tiempoFinalEscribirGameFeaturesCSV)
tiempoTotalReduccionArchivoGameFeatures = time() - tiempoInicialReduccion 
print("Tiempo total de ejecucion para tratar y guardar GameFeatures = ", tiempoTotalReduccionArchivoGameFeatures)
requiredID = newGameFeatures['ResponseID']#Id de los juegos que se seleccionaran en el steam reviews
print("Cantidad de juegos disponibles en GameFeaturesFinal: ",len(requiredID))

#Ahora trataremos el steamReviews
tiempoInicialTratarSteamReviews = time()

reviewsGameExistInFeatures = pd.DataFrame(columns=esrdf_columns)
for id in requiredID:
    rows = esrdf.query('app_id == '+str(id))
    reviewsGameExistInFeatures = reviewsGameExistInFeatures.append(rows)
    
toDeleteReviews = ['Unnamed: 0', 'weighted_vote_score', 'author.last_played']
for to in toDeleteReviews:
    del reviewsGameExistInFeatures[to]

#Libera la memoria.
del [[esrdf]]
gc.collect()

tiempoFinalTratarSteamReviews = time() - tiempoInicialTratarSteamReviews
print("Tiempo que tomo generar nuevo data frame con las reviews de los juegos que existen en game features = ", tiempoFinalTratarSteamReviews)


tiempoInicialEscribirReviews = time()
reviewsGameExistInFeatures.to_csv('final-to-analice-feelings-s-reviews.csv',index=False)
tiempoFinalEscribirReviews = time() - tiempoInicialEscribirReviews
print("Tiempo que tomo escribir el nuevo steam reviews = ", tiempoFinalEscribirReviews)

tiempoFinalTotalSteamReviews = time() - tiempoInicialTratarSteamReviews
print("Tiempo total de tratamiento a steam reviews = ", tiempoFinalTotalSteamReviews)
tiempoReduccion2 = time() - tiempoInicialReduccion
print("Tiempo total reduccion ambos archivos = ", tiempoReduccion2)

#Libera la memoria.
del [[reviewsGameExistInFeatures]]
del [[newGameFeatures]]
gc.collect()

tiempoFinal = time()
tiempoTotal = tiempoFinal - tiempoInicialCruzamiento
print("Tiempo total de ejecucion: ", tiempoTotal)

# Resultado 

In [2]:
import pandas as pd

gameFeatures = pd.read_csv('final-game-features.csv', delimiter=",")
steamReviews = pd.read_csv('final-to-analice-feelings-s-reviews.csv', delimiter=",")

In [3]:
steamReviews

,QueryID,ResponseID,QueryName,ResponseName,ReleaseDate,RequiredAge,DLCCount,Metacritic,RecommendationCount,ScreenshotCount,...,AboutText,ShortDescrip,DetailedDescrip,Reviews,PCMinReqsText,PCRecReqsText,LinuxMinReqsText,LinuxRecReqsText,MacMinReqsText,MacRecReqsText
0,292030,292030,The Witcher 3: Wild Hunt,The Witcher(r) 3: Wild Hunt,May 18 2015,0,19,93,50325,14,...,The Witcher: Wild Hunt is a story-driven next-...,The Witcher is a story-driven next-generation ...,Special OfferAbout the GameThe Witcher: Wild H...,ONE OF THE BEST GAMES EVER MADE10/10 - GameSpo...,Minimum:OS: 64-bit Windows 7 64-bit Windows 8 ...,Recommended:OS: 64-bit Windows 7 64-bit Window...,,,,
1,70,70,Half-Life,Half-Life,Nov 8 1998,0,1,96,12486,11,...,Named Game of the Year by over 50 publications...,,Named Game of the Year by over 50 publications...,,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
2,240,240,Counter-Strike: Source,Counter-Strike: Source,Nov 1 2004,0,0,88,53931,5,...,THE NEXT INSTALLMENT OF THE WORLDS # 1 ONLINE ...,Just updated to include player stats achieveme...,THE NEXT INSTALLMENT OF THE WORLDS # 1 ONLINE ...,,Minimum: 1.7 GHz Processor 512MB RAM DirectX® ...,,Minimum: 1.7 GHz Processor 512MB RAM Mouse Key...,,Minimum: OS X version Leopard 10.5.8 Snow Leop...,
3,420,420,Half-Life 2: Episode Two,Half-Life 2: Episode Two,Oct 10 2007,0,1,90,6104,5,...,Half-Life® 2: Episode Two is the second in a t...,Supports cross-platform Steam Cloud - continue...,Half-Life® 2: Episode Two is the second in a t...,,Minimum: 1.7 GHz Processor 512MB RAM DirectX® ...,,,,Minimum: OS X version Leopard 10.5.8 Snow Leop...,
4,620,620,Portal 2,Portal 2,Apr 18 2011,0,1,95,73128,12,...,Portal 2 draws from the award-winning formula ...,The Perpetual Testing Initiative has been expa...,Portal 2 draws from the award-winning formula ...,,Minimum:OS: Windows 7 / Vista / XPProcessor: 3...,,Minimum:OS: Ubuntu 12.04Processor: Dual core f...,,Minimum:OS: MAC OS X 10.6.7 or higherProcessor...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,466560,466560,Northgard,Northgard,2016,0,0,0,0,8,...,After years of tireless explorations brave Vik...,Northgard is a strategy game based on Norse my...,After years of tireless explorations brave Vik...,,Minimum:OS: Windows Vista or betterProcessor: ...,,,,,
138,526160,526160,The Wild Eight,The Wild Eight,Fall 2016,0,0,0,0,6,...,The game tells the story of eight people whose...,The Wild Eight is a unforgiving co-op survival...,The game tells the story of eight people whose...,,Minimum:OS: Windows 7 or higher 64bitProcessor...,,,,,
139,312660,312660,Sniper Elite 4,Sniper Elite 4,Feb 14th 2017,0,0,0,0,13,...,Bigger and deadlier than ever before - GamesRa...,Set in the breathtaking landscapes of Italy in...,Bigger and deadlier than ever before - GamesRa...,,,,,,,
140,447820,447820,Day of Infamy,Day of Infamy,Jul 28 2016,0,0,0,1207,9,...,Experience intense close-quarters battles in i...,Experience intense close-quarters battles in i...,Experience intense close-quarters battles in i...,,Minimum:OS: Windows 7Processor: Intel(r) Core(...,Recommended:OS: Windows 10Processor: 4+ coresM...,,,Minimum:OS: MacOS X 10.7 or higherProcessor: I...,Recommended:OS: MacOS X 10.7 or higherProcesso...


In [4]:
gameFeatures

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
1,292030,The Witcher 3: Wild Hunt,85184171,english,"good story, good graphics. lots to do.",1611379264,1611379264,True,0,0,...,True,False,False,76561198170193529,11,1,823.0,823.0,823.0,1.611379e+09
2,292030,The Witcher 3: Wild Hunt,85184064,english,"dis gud,",1611379091,1611379091,True,0,0,...,True,False,False,76561198119302812,27,2,4192.0,3398.0,4192.0,1.611352e+09
3,292030,The Witcher 3: Wild Hunt,85180436,english,favorite game of all time cant wait for the Ne...,1611373086,1611373086,True,0,0,...,True,False,False,76561198065591528,33,1,23329.0,177.0,23329.0,1.611219e+09
4,292030,The Witcher 3: Wild Hunt,85179753,english,Why wouldn't you get this,1611371978,1611371978,True,0,0,...,True,False,False,76561198996835044,131,2,8557.0,2004.0,8557.0,1.611371e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6765938,512900,Streets of Rogue,30438766,english,What a great game! \n\nFor a while I was looki...,1489205145,1489205145,True,1,0,...,True,False,True,76561198066297673,296,6,7686.0,0.0,869.0,1.608692e+09
6765939,512900,Streets of Rogue,85149009,english,Very good and fast-paced roguelite. \n\nYou pi...,1611329631,1611329631,True,0,0,...,True,False,False,76561198223212980,389,6,6602.0,27.0,6602.0,1.611184e+09
6765940,512900,Streets of Rogue,85208044,english,My stand up comedy was so good it overthrew th...,1611413586,1611413685,True,0,0,...,True,False,False,76561198299024215,67,41,1086.0,745.0,1086.0,1.611413e+09
6765941,512900,Streets of Rogue,85187860,english,i went ape mode on civilians as an ape.,1611385439,1611385439,True,0,0,...,False,False,False,76561198981467371,55,16,193.0,76.0,193.0,1.611381e+09


In [3]:
def tiempoConvSTR(tSegundos):
    hor=(int(tSegundos/3600))
    minu=int((tSegundos-(hor*3600))/60)
    seg=tSegundos-((hor*3600)+(minu*60))
    return str(hor)+"h "+str(minu)+"m "+str(seg)+"s"

print('Tiempo de demora en leer los archivos : ' ,tiempoConvSTR(61.190919399261475))
print('Tiempo que tomo reducir el archivo GameFeatures : ' ,tiempoConvSTR(11.98086929321289))
print('Tiempo de escritura de nuevo GameFeaturesCSV : ' ,tiempoConvSTR(0.05996847152709961))
print('Tiempo total de ejecucion para tratar y guardar GameFeatures : ' ,tiempoConvSTR(12.04083776473999))
print('Tiempo que tomo generar nuevo data frame con las reviews de los juegos que existen en game features : ' ,tiempoConvSTR(372.47310996055603))
print('Tiempo que tomo escribir el nuevo steam reviews : ' ,tiempoConvSTR(67.56201195716858))
print('Tiempo total de tratamiento a steam reviews : ' ,tiempoConvSTR(440.0351219177246))
print('Tiempo total reduccion ambos archivos : ' ,tiempoConvSTR(452.0769603252411))
print('Tiempo total de ejecucion : ' ,tiempoConvSTR(513.2678797245026))

Tiempo de demora en leer los archivos :  0h 1m 1.1909193992614746s
Tiempo que tomo reducir el archivo GameFeatures :  0h 0m 11.98086929321289s
Tiempo de escritura de nuevo GameFeaturesCSV :  0h 0m 0.05996847152709961s
Tiempo total de ejecucion para tratar y guardar GameFeatures :  0h 0m 12.04083776473999s
Tiempo que tomo generar nuevo data frame con las reviews de los juegos que existen en game features :  0h 6m 12.47310996055603s
Tiempo que tomo escribir el nuevo steam reviews :  0h 1m 7.562011957168579s
Tiempo total de tratamiento a steam reviews :  0h 7m 20.03512191772461s
Tiempo total reduccion ambos archivos :  0h 7m 32.07696032524109s
Tiempo total de ejecucion :  0h 8m 33.26787972450256s


In [2]:
text = 61.190919399261475
print()

Index(['app_id', 'app_name', 'review_id', 'language', 'review',
       'timestamp_created', 'timestamp_updated', 'recommended',
       'votes_helpful', 'votes_funny', 'comment_count', 'steam_purchase',
       'received_for_free', 'written_during_early_access', 'author.steamid',
       'author.num_games_owned', 'author.num_reviews',
       'author.playtime_forever', 'author.playtime_last_two_weeks',
       'author.playtime_at_review', 'author.last_played'],
      dtype='object')